$$V_t=\frac{2}{\alpha \Delta t}\left[\frac{S_{t+\Delta t}^{1+\alpha}-S_t^{1+\alpha}}{(1+\alpha) S_t^{1+\alpha}}-\frac{S_{t+\Delta t}-S_t}{S_t}\right]$$
$$\alpha=-\frac{13}{11}-\frac{12}{11} \frac{\mu}{\sigma\left(S_t\right)^2}$$
$$\ln \sigma\left(S_t\right)^2 =  2 \ln (\delta) + (\theta -2) \ln (S_t)$$
$$\sum_{t=1}^{n-1}\left(\ln V_t-\ln \sigma\left(S_t\right)^2\right)^2$$
所以$\ln V_t$对$\ln \left(S_t\right)$做线性回归，得到的系数既是$\theta -2$

In [1]:
import numpy as np

# CEV process
class ProcessCEV:

    def __init__(self, mu, sigma, gamma):
        self._mu = mu
        self._sigma = sigma
        self._gamma = gamma

    def Simulate(self, T=1, dt=0.001, S0=1.):
        n = round(T / dt)
        
        mu = self._mu
        sigma = self._sigma
        gamma = self._gamma

        gaussian_increments = np.random.normal(size=n - 1)
        res = np.zeros(n)
        res[0] = S0
        S = S0
        sqrt_dt = dt ** 0.5
        for i in range(n - 1):
            S = S + S * mu * dt + sigma * \
                (S ** gamma) * gaussian_increments[i] * sqrt_dt
            res[i + 1] = S

        return res
    
class EstimatorCEV:

    def __init__(self, dt):
        self._dt = dt
        self._alpha0 = -5
    '''
    trajectory:S_t
    V_T is an estimate of  sigma^2 (delta^2 S^{theta-2})
    '''
    def Estimate(self, trajectory):
        sigma, gamma = self._evaluate_sigma_gamma(trajectory, self._alpha0)
        if sigma == None:
            return None, None, None
        else:
            mu = self._estimate_mu(trajectory)
            return (mu, sigma, gamma)

    def _log_increments(self, trajectory):# 除以差分，类似于收益率
        return np.diff(trajectory) / trajectory[:-1]
    
    def _estimate_mu(self, trajectory):# 涨跌幅除以时间，就得到mu
        return np.mean(self._log_increments(trajectory)) / self._dt 

    def _log_increments_alpha(self, trajectory, alpha):#计算V_t的第一项
        mod_increments = self._log_increments(trajectory ** (1 + alpha))
        return mod_increments / (1 + alpha)

    def _evaluate_Vt(self, trajectory, alpha):#计算V_t
        lhs = self._log_increments_alpha(trajectory, alpha)
        rhs = self._log_increments(trajectory)# V_t第二项
        center = 2 * (lhs - rhs) / (alpha * self._dt)
        return center

    def _evaluate_sigma_gamma(self, trajectory, alpha):
        if np.any(trajectory <= 0):
            return None, None
        
        Vts = self._evaluate_Vt(trajectory, alpha)
        if np.any(Vts <= 0):
            return None, None
        logVts = np.log(Vts)

        Sts = trajectory[:-1]  # removes the last term as in eq. (5)
        if np.any(Sts <= 0):
            return None, None
        logSts = np.log(Sts)

        ones = np.ones(Sts.shape[0])
        A = np.column_stack((ones, logSts)) 

        res = np.linalg.lstsq(A, logVts, rcond=None)[0]
        return (2 * np.exp(res[0] / 2), 0.5 * (res[1] + 2))


if __name__ == "__main__":

    #from ProcessCEV import ProcessCEV

    def test(true_mu, true_sigma, true_gamma):
        dt = 0.001
        T = 10
        
        sample_mu = []
        sample_sigma = []
        sample_gamma = []

        for i in range(100):
            mu_est, sigma_est, gamma_est = EstimatorCEV(dt=dt).Estimate(ProcessCEV(
                true_mu, true_sigma, true_gamma).Simulate(T, dt=dt))
            
            if mu_est != None:
                sample_mu = [mu_est] + sample_mu
                sample_sigma = [sigma_est] + sample_sigma
                sample_gamma = [gamma_est ] + sample_gamma
    
        print("mu : " + str(true_mu) + " \t| est : " + str(np.mean(sample_mu)) + " \t| std : " + str(np.std(sample_mu)))
        print("sigma : " + str(true_sigma) + " \t| est : " + str(np.mean(sample_sigma)) + " \t| std : " + str(np.std(sample_sigma)))
        print("gamma : " + str(true_gamma) + " \t| est : " + str(np.mean(sample_gamma)) + " \t| std : " + str(np.std(sample_gamma)))
        print(10*"-")


    test(0.,0.5,0.8)
    test(0.2,0.5,0.8)
    test(0.2,0.5,1.2)
    test(0.,0.3,0.2)
    test(0.,0.5,2)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_8368\4270833563.py:25: RuntimeWarning: invalid value encountered in scalar power
  (S ** gamma) * gaussian_increments[i] * sqrt_dt


mu : 0.0 	| est : 0.0546671592477858 	| std : 0.12612995439664326
sigma : 0.5 	| est : 0.5317858190187142 	| std : 0.009369757610233294
gamma : 0.8 	| est : 0.7992122042086476 	| std : 0.02075882034205364
----------
mu : 0.2 	| est : 0.21256521457261549 	| std : 0.1317677726684672
sigma : 0.5 	| est : 0.5293440511669173 	| std : 0.012143313415460853
gamma : 0.8 	| est : 0.8017122562248804 	| std : 0.023680957060982364
----------
mu : 0.2 	| est : 0.21252535721552476 	| std : 0.20492531692354116
sigma : 0.5 	| est : 0.5298720469333866 	| std : 0.007976794683717339
gamma : 1.2 	| est : 1.202080714625498 	| std : 0.02217161808579669
----------
mu : 0.0 	| est : 0.07460335526134648 	| std : 0.10189909088703278
sigma : 0.3 	| est : 0.31747647885466007 	| std : 0.005093005173148378
gamma : 0.2 	| est : 0.19954500356577798 	| std : 0.044427130534360294
----------
mu : 0.0 	| est : -0.0031449500061706257 	| std : 0.11175282107770093
sigma : 0.5 	| est : 0.530992981815231 	| std : 0.01216946650

In [26]:
len(ProcessCEV(0.2,0.5,0.8).Simulate(10, dt=0.001))#模拟CEV模型下的股价走势

10000

In [37]:
dt = 0.001
T = 10
trajectory=ProcessCEV(0.2,0.5,0.8).Simulate(T, dt=dt)
Vts = EstimatorCEV(dt=dt)._evaluate_Vt(trajectory, -5)
logVts = np.log(Vts)

Sts = trajectory[:-1]  # removes the last term as in eq. (5)

logSts = np.log(Sts)

ones = np.ones(Sts.shape[0])
A = np.column_stack((ones, logSts)) 

res = np.linalg.lstsq(A, logVts, rcond=None)[0]

In [44]:
np.linalg.lstsq(A, logVts, rcond=None)

(array([-2.66660035, -0.43004486]),
 array([51328.95374597]),
 2,
 array([122.73364721,  43.23263573]))

In [38]:
len(A)

9999

In [36]:
len(Vts)

9999

In [30]:
np.linalg.lstsq(A, logVts, rcond=None)[0]

NameError: name 'logVts' is not defined

In [21]:
ones

array([1., 1.])

In [22]:
A

array([[1.        , 0.        , 0.69314718, 3.4657359 , 1.09861229,
        0.69314718, 1.09861229, 1.38629436, 3.52636052, 1.09861229,
        3.73766962],
       [1.        , 0.        , 0.69314718, 3.4657359 , 1.09861229,
        0.69314718, 1.09861229, 1.38629436, 3.52636052, 1.09861229,
        3.73766962]])